In [ ]:
# コスト更新するシート

In [ ]:
########################
# 
# 設定（変動）
# 
########################

# 該当日時
this_year  = 2022  # 実行年
this_month = 7 # 実行月

In [ ]:
########################
# 
# モジュール
# 
########################

# # 一般モジュール
import time
import datetime as dt
import pytz
import glob
import calendar

# import pandas as pd
# import re
# import pytz
# from datetime import datetime as dt

# import os
# import csv
# import time
from selenium.webdriver.common.by import By

# # スプシ関連
# from gspread_dataframe import set_with_dataframe

# 自作モジュール
import func
import conf

print("モジュールのインポート完了")

In [179]:
########################
# 
# 設定（固定）
# 
########################

# 実行時間
now = dt.datetime.now(pytz.timezone('Asia/Tokyo'))

# ログイン情報（基幹）
url_kikan = "https://mensclear.com/users/reservations" # URL
email_kikan = "cb140000@yahoo.co.jp" # メールアドレス
pass_kikan = "muramura" # パスワード

# ログイン情報（レントラ）
url_rentra = "https://manage.rentracks.jp/sponsor/" # URL
email_rentra = "sem@mk-force.com" # メールアドレス
pass_rentra = "@F4Czu%5" # パスワード

# ログイン情報（クオリザ）
url_quoriza = "https://quoriza.net/client/login" # URL
id_quoriza = "9N7PMJ5j" # メールアドレス
pass_quoriza = "faclear2019" # パスワード

# 実行年月日取得
exe_year = now.year # 実行年
exe_month = "0"+str(now.month) if len(str(now.month))==1 else str(now.month) # 実行月
exe_day = "0"+str(now.day) if len(str(now.day))==1 else str(now.day) # 実行日

# 実行月と該当月が一致するか
is_this_month = True if exe_month==this_month else False

print("変数の設定完了")



# month_of_bigin = "01"
# month_of_end = calendar.monthrange(this_year, this_month)[1]
# this_month_rentra = "0"+str(this_month) if len(str(this_month))==1 else str(this_month)
# 実行月

# # CSVファイルの保存先
# csv_filepass = "data/reservations.csv"

# # 実行月
# this_month = dt.today().month

変数の設定完了


In [ ]:
########################
# 
# 関数
# 
########################

# ダウンロード完了するまで監視する関数
def Monitor_download(file_pass):
    for ind in range(0,120):
        file = glob.glob(file_pass)
        if file:
            print("ダウンロード完了")
            break
        else:
            time.sleep(1)

In [ ]:
########################
# 
# クローム起動
# 
########################
driver = func.start_chrome(False)

In [ ]:
########################
# 
# 基幹データダウンロード
# 
########################

# 基幹システムにアクセス
driver.get(url_kikan)
time.sleep(3)

# メールに入力
elm_email = driver.find_element(By.ID, "user_email")
elm_email.send_keys(email_kikan)
time.sleep(1)

# パスワードを入力
elm_pass = driver.find_element(By.ID, "user_password")
elm_pass.send_keys(pass_kikan)
time.sleep(1)

# ログインボタン押下
elm_login_btn = driver.find_element(By.CLASS_NAME, "p-sessions__button")
elm_login_btn.click()
time.sleep(3)
print("基幹システムにログイン")

# データ一覧を表示
elm_data_index = driver.find_elements(By.CLASS_NAME, "l-dashboard__panel")[0]
elm_data_index.click()
time.sleep(3)

# 今年を選択
elm_year_list = driver.find_element(By.ID, "_date_1i")
elms_year = elm_year_list.find_elements(By.TAG_NAME, "option")
for elm in elms_year:
    val = elm.text
    if(val == str(this_year)):
        elm.click()
        time.sleep(1)
        break

# 今月を選択
elm_month_list = driver.find_element(By.ID, "_date_2i")
elms_month = elm_month_list.find_elements(By.TAG_NAME, "option")
for elm in elms_month:
    val = elm.text
    if(val == str(this_month)):
        elm.click()
        time.sleep(1)
        break

# ダウンロードボタンをクリック
elm_dl_btn = driver.find_element(By.NAME, "commit")
elm_dl_btn.click()

# ダウンロード完了するまで待機
Monitor_download("data/reservations.csv")

In [ ]:
########################
# 
# レントラデータダウンロード
# 
########################

# レントラ管理画面にアクセス
driver.get(url_rentra)
time.sleep(3)

# メールに入力
elm_email = driver.find_element(By.NAME, "idMailaddress")
elm_email.send_keys(email_rentra)
time.sleep(1)

# パスワードを入力
elm_pass = driver.find_element(By.NAME, "idLoginPassword")
elm_pass.send_keys(pass_rentra)
time.sleep(1)

# ログインボタン押下
elm_login_btn = driver.find_element(By.NAME, "idButton")
elm_login_btn.click()
time.sleep(3)
print("レントラ管理画面にログイン")

# 注文リストをクリック
elm_data_index = driver.find_element(By.ID, "hnavInner").find_elements(By.TAG_NAME, "li")[2]
elm_data_index.click()
time.sleep(3)

# 該当月を選択
elm_month_select = driver.find_element(By.ID, "idTermSelect")
if is_this_month:
    elm_month_select.send_keys("今月")
else:
    elm_month_select.send_keys("先月")

# 保留、承認済、請求済を選択
# チェックを入れる関数
def check_elm(name):
    driver.execute_script("\
        const elm = document.getElementsByName('" + name + "')[0];\
        elm.setAttribute('checked', 'checked');\
    ")
check_elm("idFilter1") # 保留
check_elm("idFilter3") # 承認済
check_elm("idFilter4") # 請求済

# 再表示をクリック
btn_reload = driver.find_element(By.NAME, "idButton1")
btn_reload.click()
time.sleep(2)

# ダウンロードボタンクリック
elm_btn_download = driver.find_element(By.ID, "idButtonFD")
elm_btn_download.click()
Monitor_download("data/data.csv")

In [ ]:
########################
# 
# クオリザデータダウンロード
# 
########################

# レントラ管理画面にアクセス
driver.get(url_quoriza)
time.sleep(3)

# メールに入力
elm_email = driver.find_element(By.NAME, "c_login_name")
elm_email.send_keys(id_quoriza)
time.sleep(1)

# パスワードを入力
elm_pass = driver.find_element(By.NAME, "c_login_password")
elm_pass.send_keys(pass_quoriza)
time.sleep(1)

# ログインボタン押下
elm_login_btn = driver.find_element(By.NAME, "clientlogin")
elm_login_btn.click()
time.sleep(3)
print("クオリザ管理画面にログイン")

# コンバージョン一覧画面に遷移
driver.get('https://quoriza.net/client/conversion/tracking')
time.sleep(3)

# 該当月のボタンをクリック
elm_form_inline = driver.find_element(By.CLASS_NAME, "form-inline")
elms_select_month = elm_form_inline.find_elements(By.TAG_NAME, "a")
if is_this_month:
    elms_select_month[0].click()
else:
    elms_select_month[1].click()
time.sleep(1)

# ダウンロードボタンをクリック
elm_btn_download = driver.find_element(By.XPATH, '//*[@id="view"]/div/button[2]')
elm_btn_download.click()
Monitor_download("data/conversion.tracking." + str(exe_year) + str(exe_month) + str(exe_day) + ".csv")